## Handling Missing Data
## 處理缺失數據

> The difference between data found in many tutorials and data in the real world is that real-world data is rarely clean and homogeneous. In particular, many interesting datasets will have some amount of data missing.
To make matters even more complicated, different data sources may indicate missing data in different ways. we'll refer to missing data in general as *null*, *NaN*, or *NA* values.

我們在許多教程裡面看到的數據和真實的數據的區別就是真實的數據很少是乾淨和同質的。更尋常的情況是，很多有意思的數據集都有很多的數據缺失。更複雜的是，不同的數據源可能有著不同指代缺失數據的方式，我們會將這些缺失數據標示為*null*、*NaN*或*NA*。

### ``None``: Pythonic missing data 缺失值
> The first sentinel value used by Pandas is ``None``, a Python singleton object that is often used for missing data in Python code.
Because it is a Python object, ``None`` cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type ``'object'`` (i.e., arrays of Python objects):

第一個種缺失值是 None，很多情況下它都作為 Python 代碼中缺失值的標誌。None 不能被任意的 NumPy 或Pandas 數組中使用，當使用聚合操作如sum()或min()的時候，如果碰到了None值，那就會產生錯誤：

In [1]:
import numpy as np
import pandas as pd

vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

In [3]:
#vals1.sum()

### ``NaN``: Missing numerical data 缺失值

> The other missing data representation, ``NaN`` (acronym for *Not a Number*), is different; Notice that NumPy chose a native floating-point type for this array: this array supports fast operations pushed into compiled code.You should be aware that ``NaN`` is a bit like a data virus–it infects any other object it touches.Regardless of the operation, the result of arithmetic with ``NaN`` will be another ``NaN``, NumPy does provide some special aggregations that will ignore these missing values: Keep in mind that ``NaN`` is specifically a floating-point value; there is no equivalent NaN value for integers, strings, or other types.

另外一個缺失的數據表現形式`NaN`（*非數字*的縮寫），NumPy使用原始的浮點類型來存儲這個數組：這個數組支持使用編譯代碼來進行快速運算。你應該了解到`NaN`就像一個數據的病毒，它會傳染到任何接觸到的數據。不論運算是哪種類型，`NaN`參與的算術運算的結果都會是另一個`NaN`，請記住`NaN`是一個特殊的浮點數值；對於整數、字符串或者其他類型來說都沒有對應的值。

In [4]:
1 + np.nan
0 * np.nan

nan

In [6]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2

array([ 1., nan,  3.,  4.])

In [7]:
vals2.sum(), vals2.min(), vals2.max()  #

(nan, nan, nan)

In [8]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)   #

(8.0, 1.0, 4.0)

## Operating on Null Values
## 操作空值

> Pandas treats ``None`` and ``NaN`` as essentially interchangeable for indicating missing or null values.
To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in Pandas data structures.
They are:

Pandas將`None`和`NaN`看成是可以互相轉換的缺失值或空值。與此同時，Pandas還提供了一些很有用的方法用來在數據集中發現、移除和替換空值。這些方法包括：

- `isnull()`：生成一個布爾遮蓋數組指示缺失值的位置
- `notnull()`：`isnull()`相反方法
- `dropna()`：返回一個過濾掉缺失值、空值的數據集
- `fillna()`：返回一個數據集的副本，裡面的缺失值、空值使用另外的值來替代

### isnull() - Detecting null values 檢測空值

In [3]:
data = pd.Series([1, np.nan, 'hello', None])
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

### notnull() - Detecting null values 檢測空值

In [4]:
data = pd.Series([1, np.nan, 'hello', None])
data[data.notnull()]

0        1
2    hello
dtype: object

### dropna() - Dropping null values 去除空值

In [5]:
data = pd.Series([1, np.nan, 'hello', None])
data.dropna()

0        1
2    hello
dtype: object

### isnull() - Filling null values 填充空值

In [6]:
data = pd.Series([1, np.nan, 'hello', None])
data.fillna(0)  # fill NA entries with a single value
data.fillna(method='ffill') # 向前填充  forward-fill
data.fillna(method='bfill') # 向後填充 back-fill

0        1
1        0
2    hello
3        0
dtype: object

## DataFrame

> We cannot drop single values from a ``DataFrame``; we can only drop full rows or full columns.
Depending on the application, so ``dropna()`` gives a number of options for a ``DataFrame``.By default, ``dropna()`` will drop all rows in which *any* null value is present: Alternatively, you can drop NA values along a different axis; ``axis=1`` drops all columns containing a null value:

我們不能在`DataFrame`中移除單個空值；我們只能移除整行或者整列。`dropna()`為`DataFrame`對象提供了一些參數選擇。默認，`dropna()`會移除出現了空值的整行：你可以通過設置axis參數（如`axis=1`）來沿著不同的維度來移除空值，下面是移除含有空值的列的例子：

In [13]:
import pandas as pd
import numpy as np

df = pd.DataFrame([[1,      np.nan, 2],[2,      3,      5],[np.nan, 4,      6]],columns=list('ABC'))
df

,A,B,C
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [14]:
df.dropna()                #移除整行 default
df.dropna(axis='rows')     #移除整行
df.dropna(axis='columns')  #移除整列
df.dropna(axis='rows', thresh=3)  #行中如果有3個或以上的非空值，將會被保留
df.dropna(axis='columns', how='all')  #行或列全部由空值構成的情況下才會被移除
df.dropna(axis='columns', how='any')  #行或列只要含有空值都會被移除

,C
0,2
1,5
2,6


In [15]:
df.fillna(50)
df.fillna(method='ffill')
df.fillna(method='bfill')
df.fillna(method='ffill', axis=1)  # 按列進行向前填充
df.fillna(method='ffill', limit=2)

,A,B,C
0,1.0,NaN,2
1,2.0,3.0,5
2,2.0,4.0,6


## Handling missing values in dataframes

Missing Data can occur when no information is provided for one or more items or for a whole unit. Missing Data is a very big problem in real life scenario.

Missing Data can also refer to as NA(Not Available) values in pandas. In DataFrame sometimes many datasets simply arrive with missing data, either because it exists and was not collected or it never existed.

For Example, Suppose different user being surveyed may choose not to share their income, some user may choose not to share the address in this way many datasets went missing.

* None: None is a Python singleton object that is often used for missing data in Python code.
* NaN : NaN (an acronym for Not a Number), is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation.

Pandas treat None and NaN as essentially interchangeable for indicating missing or null values. To facilitate this convention, there are several useful functions for detecting, removing, and replacing null values in Pandas DataFrame :

* isnull()
* notnull()
* dropna()
* fillna()
* replace()
* interpolate()

In [36]:
# importing libraries
import pandas as pd
import numpy as np
# creating dataframe
d = {'First Score':[100, 90, np.nan, 95],
        'Second Score': [30, 45, 56, np.nan],
        'Third Score':[np.nan, 40, 80, 98]}
df = pd.DataFrame(d)
df

,First Score,Second Score,Third Score
0,100.0,30.0,NaN
1,90.0,45.0,40.0
2,NaN,56.0,80.0
3,95.0,NaN,98.0


In [37]:
df.isnull().sum()

First Score     1
Second Score    1
Third Score     1
dtype: int64

In [38]:
df.isnull().sum(axis = 1)

0    1
1    0
2    1
3    1
dtype: int64

In [39]:
df.fillna(df.mean())

,First Score,Second Score,Third Score
0,100.0,30.000000,72.666667
1,90.0,45.000000,40.000000
2,95.0,56.000000,80.000000
3,95.0,43.666667,98.000000


In [40]:
# importing required libraries
import pandas as pd
import numpy as np
d = {"col1": [2019, 2019, 2020],
     "col2": [350, 365, 1],
     "col3": [np.nan, 365, None]}

df = pd.DataFrame(d)
df

,col1,col2,col3
0,2019,350,NaN
1,2019,365,365.0
2,2020,1,NaN


In [41]:
df.isnull().sum() # Solution 1
df.isna().sum()   # Solution 2
df.isna().any()  # Solution 3
df.isna().sum(axis = 1) # Solution 4

0    1
1    0
2    1
dtype: int64

In [42]:
# total number of missing values in the dataframe
df.isnull().sum().sum()

2

In [43]:
# rowwise missing values
df.isnull().sum(axis=1)

0    1
1    0
2    1
dtype: int64

In [44]:
# returns boolean object 
df.isnull().any()

col1    False
col2    False
col3     True
dtype: bool

<!--NAVIGATION-->
< [在Pandas中操作数据](03.03-Operations-in-Pandas.ipynb) | [目录](Index.ipynb) | [层次化的索引](03.05-Hierarchical-Indexing.ipynb) >

<a href="https://colab.research.google.com/github/wangyingsm/Python-Data-Science-Handbook/blob/master/notebooks/03.04-Missing-Values.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
